# LangGraph Components

In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [14]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
# from langchain_ollama import ChatOllama as ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


> If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name. Even with function calling, LLMs can still occasionally hallucinate. Note that all that is done is instructing the LLM to try again! An advantage of an agentic organization.

In [5]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="llama3.2")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [7]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '939fc765-59c4-4dd4-9c24-f49c2d2a29c9', 'type': 'tool_call'}
Back to the model!


In [8]:
result

{'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-10-18T05:10:32.7644737Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'sf weather'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 4192388900, 'load_duration': 3658318300, 'prompt_eval_count': 258, 'prompt_eval_duration': 224895000, 'eval_count': 22, 'eval_duration': 301408000}, id='run-72d1bd5d-e955-45e2-a949-17377dc5fc60-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '939fc765-59c4-4dd4-9c24-f49c2d2a29c9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 258, 'output_tokens': 22, 'total_tokens': 280}),
  ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San F

In [9]:
result['messages'][-1].content

'The current weather in San Francisco is clear with a temperature of 16.1°C (61.0°F). The wind is blowing at 14.3 mph (23.0 km/h) from the north, and the pressure is 1014.0 mb. There is no precipitation expected, and the humidity is at 62%.'

In [10]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today and Los Angeles weather today'}, 'id': '8b10e8c6-60c3-42e6-9627-b5cc111c237f', 'type': 'tool_call'}
Back to the model!


In [11]:
result['messages'][-1].content

"Unfortunately, I was only able to retrieve information on the weather in LA. For SF, the available tool call results were incomplete.\n\nHowever, based on the provided search results, here is what we can infer about the current weather in Los Angeles:\n\nThe current weather conditions in Los Angeles are mostly cloudy with a temperature of 60.1°F (15.6°C). There is no precipitation reported, and the wind speed is 6.9 mph (11.2 km/h) from the northwest.\n\nAs for San Francisco, the available tool call results were incomplete, but it appears that the forecast predicts equal chances of above- and below-normal precipitation in California in winter 2024-25 due to a weak La Niña forecast. Additionally, NOAA predicts that San Francisco's normal December through February temperatures will be around 18.5°C (65.3°F) during the day.\n\nIt's worth noting that these results are subject to change and may not reflect the current weather conditions in either location. For more up-to-date information, 

In [15]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "How was the trading session in NYSE yesterday (17 October 2024) and what was the mood of the market? What may we expect for today? Do we have a significant news event today?" 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'NYSE trading session October 17 2024'}, 'id': 'call_IzEncWb0YUwxQuMzedw0SY5n', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'market outlook October 18 2024'}, 'id': 'call_9Bac2W6Rpug0pp9IYLTyaQSM', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'significant news events October 18 2024'}, 'id': 'call_hdjpdLNpvnvOd57Kz7fKn0g8', 'type': 'tool_call'}
Back to the model!


In [16]:
print(result['messages'][-1].content)

### NYSE Trading Session on October 17, 2024

The trading session on October 17, 2024, saw advancers outnumbering decliners on the NYSE by a 2.38-to-1 ratio. This indicates a generally positive sentiment in the market as more stocks were advancing than declining. The trading volume was slightly lower than the average, but the overall mood seemed optimistic.

- Source: [Nasdaq](https://www.nasdaq.com/articles/stock-market-news-oct-17-2024)

### Market Outlook for October 18, 2024

The market outlook for today, October 18, 2024, is cautiously optimistic. With recent strong gains in stocks and bonds, investors seem hopeful about the end of the Federal Reserve's rate-hiking cycle. However, it's important to note that upcoming U.S. presidential elections and geopolitical risks may introduce volatility in the near term.

- Source: [Fidelity](https://www.fidelity.com/learning-center/trading-investing/stock-market-outlook), [Investopedia](https://www.investopedia.com/what-to-expect-stock-marke